In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras import backend as K

# Define the VAE parameters
input_dim = 32      # Input dimension
hidden_dim = 16     # Hidden layer dimension
latent_dim = 8      # Latent dimension

# Encoder
inputs = Input(shape=(input_dim,))
h_q = Dense(hidden_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h_q)           # Mean of the latent space
z_log_var = Dense(latent_dim)(h_q)        # Log variance of the latent space

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Latent space sampling
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
h_p = Dense(hidden_dim, activation='relu')(z)
outputs = Dense(input_dim, activation='sigmoid')(h_p)

# Define the VAE model
vae = Model(inputs, outputs)

# Define the VAE loss
reconstruction_loss = mse(inputs, outputs)
reconstruction_loss *= input_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Train the VAE with some random real data
real_data = np.random.rand(1000, input_dim)  # Dummy real data
vae.fit(real_data, epochs=50, batch_size=32, verbose=1)

# Define the encoder and decoder models separately for later use
encoder = Model(inputs, z_mean)

decoder_input = Input(shape=(latent_dim,))
_h_decoded = Dense(hidden_dim, activation='relu')(decoder_input)
_x_decoded = Dense(input_dim, activation='sigmoid')(_h_decoded)
decoder = Model(decoder_input, _x_decoded)

# Generate synthetic samples using the decoder
num_samples = 100
z_sample = np.random.normal(size=(num_samples, latent_dim))
synthetic_data = decoder.predict(z_sample)

# Function to perform Bayesian Inference
def bayesian_inference(data, prior_mean=0, prior_variance=1, likelihood_variance=0.5):
    # Observed data mean and variance
    observed_mean = np.mean(data)
    observed_variance = np.var(data)
    
    # Posterior mean and variance calculation using Bayesian update
    posterior_variance = 1 / (1 / prior_variance + len(data) / likelihood_variance)
    posterior_mean = posterior_variance * (prior_mean / prior_variance + len(data) * observed_mean / likelihood_variance)
    
    return posterior_mean, posterior_variance

# Inference with Real Data Only
posterior_mean_real, posterior_variance_real = bayesian_inference(real_data)

# Inference with Synthetic Data Only
posterior_mean_synthetic, posterior_variance_synthetic = bayesian_inference(synthetic_data)

# Inference with Combined Data (Real + Synthetic)
combined_data = np.vstack((real_data, synthetic_data))
posterior_mean_combined, posterior_variance_combined = bayesian_inference(combined_data)

# Display Results
print(f"Posterior Mean with Real Data: {posterior_mean_real}")
print(f"Posterior Variance with Real Data: {posterior_variance_real}\n")

print(f"Posterior Mean with Synthetic Data: {posterior_mean_synthetic}")
print(f"Posterior Variance with Synthetic Data: {posterior_variance_synthetic}\n")

print(f"Posterior Mean with Combined Data: {posterior_mean_combined}")
print(f"Posterior Variance with Combined Data: {posterior_variance_combined}\n")

# Plot the Posterior Distributions for Visual Comparison
x = np.linspace(-1, 1, 200)

# Posterior with Real Data Only
posterior_real = (1 / np.sqrt(2 * np.pi * posterior_variance_real)) * np.exp(-0.5 * (x - posterior_mean_real)**2 / posterior_variance_real)

# Posterior with Synthetic Data Only
posterior_synthetic = (1 / np.sqrt(2 * np.pi * posterior_variance_synthetic)) * np.exp(-0.5 * (x - posterior_mean_synthetic)**2 / posterior_variance_synthetic)

# Posterior with Combined Data
posterior_combined = (1 / np.sqrt(2 * np.pi * posterior_variance_combined)) * np.exp(-0.5 * (x - posterior_mean_combined)**2 / posterior_variance_combined)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(x, posterior_real, label='Posterior with Real Data', color='blue')
plt.plot(x, posterior_synthetic, label='Posterior with Synthetic Data', color='red')
plt.plot(x, posterior_combined, label='Posterior with Combined Data', color='green')
plt.xlabel("Hypothesis Mean")
plt.ylabel("Posterior Probability Density")
plt.title("Posterior Distributions with Real, Synthetic, and Combined Data")
plt.legend()
plt.show()


2024-11-02 15:17:45.578012: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
